In [1]:
## Simple LLM Workflow

In [2]:
from langgraph.graph import StateGraph, START, END
from langchain_ollama import ChatOllama
from typing import TypedDict

In [17]:
!ollama list

NAME                       ID              SIZE      MODIFIED   
gemma3:4b                  a2af6cc3eb7f    3.3 GB    5 days ago    
nomic-embed-text:latest    0a109f422b47    274 MB    8 days ago    
llama3.2:1b                baf6a787fdff    1.3 GB    8 days ago    


In [19]:
# model=ChatOllama(model="gemma3:4b")
model=ChatOllama(model="llama3.2:1b")

In [20]:
# Create a state  

class LLMState(TypedDict):
    question: str 
    answer: str
    

In [21]:
def llm_qa(state: LLMState) -> LLMState:

    # extract the question from state 
    question = state['question']

    # form the prompt 
    prompt = f"answer the follwoing question {question}"

    # ask the question to the LLM
    answer = model.invoke(prompt).content

    # update the answer in state 
    state['answer'] = str(answer)

    return state

In [22]:
graph = StateGraph(LLMState)

# add nodes 
graph.add_node('llm_qa', llm_qa)

# add edge
graph.add_edge(START, 'llm_qa')
graph.add_edge('llm_qa', END)

# compile the graph
workflow = graph.compile()


In [23]:
# exectue 

inital_state: LLMState = {'question':'How far is moon from  the earth?','answer':''}

final_state = workflow.invoke(inital_state)

print(final_state)

{'question': 'How far is moon from  the earth?', 'answer': "The average distance from the Earth to the Moon is approximately 384,400 kilometers (238,900 miles). This distance is constantly changing due to the elliptical shape of the Moon's orbit around the Earth. At its closest point, known as perigee, the distance is about 356,400 kilometers (221,500 miles), and at its farthest point, known as apogee, the distance is about 405,500 kilometers (252,000 miles)."}
